In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from scipy.stats import norm

In [2]:
# Cargar los datos desde el archivo Excel
df = pd.read_excel('C:/Users/nef_m/OneDrive/Desktop/DATOS TESIS MGO/DATOS POR MES.xlsx')

# Mostrar las primeras filas del DataFrame para verificar la estructura de los datos
print(df.head())

# Convertir la columna 'FECHA_IN' a formato datetime y establecerla como índice
df['FECHA_IN'] = pd.to_datetime(df['FECHA_IN'], format='%d-%m-%Y')
df.set_index('FECHA_IN', inplace=True)

# Mostrar el DataFrame con la columna de fecha ajustada
print(df.head())

    FECHA_IN  ALUMINIO_(USD/TON)  USD_CLP   YUAN_CLP  BARRIL_PETROLEO  TI_USA  \
0 2002-01-01         1371.363636   667.28  80.622078        19.416818  0.0175   
1 2002-02-01         1371.080000   678.84  82.030079        20.275500  0.0175   
2 2002-03-01         1404.985000   663.26  80.137981        23.696667  0.0175   
3 2002-04-01         1370.385714   650.82  78.687501        25.728636  0.0175   
4 2002-05-01         1344.427273   653.91  79.036117        25.345455  0.0175   

   IPC_USA  TON_BARRAS_ALUM_IMP  PRECIO_ALUM_IMP  VALOR_TOTAL_IMP  ...  \
0    104.2           473.456006      2878.882057       1363024.00  ...   
1    104.5           298.963345      3434.747561       1026863.62  ...   
2    105.1           579.415210      2970.067629       1720902.36  ...   
3    105.6           548.299753      2644.006079       1449707.88  ...   
4    105.6           651.638162      2959.563639       1928564.61  ...   

   VALOR_TOTAL_EXP  IMACEC    IPC  TI_CHILE   IPC_CHINA  TI_CN     I

In [3]:
# Mostrar estadísticas descriptivas
resumen = df.describe()
print(resumen)
resumen.to_excel('C:/Users/nef_m/OneDrive/Desktop/DATOS TESIS MGO/resumen.xlsx')

       ALUMINIO_(USD/TON)     USD_CLP    YUAN_CLP  BARRIL_PETROLEO  \
count          263.000000  263.000000  263.000000       263.000000   
mean          2019.908976  627.203080   89.985768        69.685439   
std            442.119723  117.709689   18.475208        27.856772   
min           1293.357143  442.940000   62.583246        18.378500   
25%           1727.574432  527.080000   74.434690        48.338636   
50%           1934.142857  618.390000   86.728118        66.138571   
75%           2316.835000  702.940000  100.633565        91.171148   
max           3498.373043  955.890000  141.603582       132.718182   

           TI_USA     IPC_USA  TON_BARRAS_ALUM_IMP  PRECIO_ALUM_IMP  \
count  263.000000  263.000000           263.000000       263.000000   
mean     0.013831  131.976445           988.576097      3611.165320   
std      0.017125   16.433312           504.352868       552.723152   
min      0.000000  104.200000           155.699708      1900.332225   
25%      0.000

In [4]:
for columna in df.columns:
    df[f'{columna}_t'] = (df[columna]-df[columna].shift(1)) / df[columna].shift(1) * 100
print(df.head())


            ALUMINIO_(USD/TON)  USD_CLP   YUAN_CLP  BARRIL_PETROLEO  TI_USA  \
FECHA_IN                                                                      
2002-01-01         1371.363636   667.28  80.622078        19.416818  0.0175   
2002-02-01         1371.080000   678.84  82.030079        20.275500  0.0175   
2002-03-01         1404.985000   663.26  80.137981        23.696667  0.0175   
2002-04-01         1370.385714   650.82  78.687501        25.728636  0.0175   
2002-05-01         1344.427273   653.91  79.036117        25.345455  0.0175   

            IPC_USA  TON_BARRAS_ALUM_IMP  PRECIO_ALUM_IMP  VALOR_TOTAL_IMP  \
FECHA_IN                                                                     
2002-01-01    104.2           473.456006      2878.882057       1363024.00   
2002-02-01    104.5           298.963345      3434.747561       1026863.62   
2002-03-01    105.1           579.415210      2970.067629       1720902.36   
2002-04-01    105.6           548.299753      2644.00607

In [5]:
resumen = df.describe()
print(resumen)

       ALUMINIO_(USD/TON)     USD_CLP    YUAN_CLP  BARRIL_PETROLEO  \
count          263.000000  263.000000  263.000000       263.000000   
mean          2019.908976  627.203080   89.985768        69.685439   
std            442.119723  117.709689   18.475208        27.856772   
min           1293.357143  442.940000   62.583246        18.378500   
25%           1727.574432  527.080000   74.434690        48.338636   
50%           1934.142857  618.390000   86.728118        66.138571   
75%           2316.835000  702.940000  100.633565        91.171148   
max           3498.373043  955.890000  141.603582       132.718182   

           TI_USA     IPC_USA  TON_BARRAS_ALUM_IMP  PRECIO_ALUM_IMP  \
count  263.000000  263.000000           263.000000       263.000000   
mean     0.013831  131.976445           988.576097      3611.165320   
std      0.017125   16.433312           504.352868       552.723152   
min      0.000000  104.200000           155.699708      1900.332225   
25%      0.000

C:\Users\nef_m\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\nanops.py:1010: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


In [6]:
df_al = pd.DataFrame({
    'PRECIO_ALUM_IMP': df['PRECIO_ALUM_IMP'],
    'PRECIO_ALUM_IMP_t': df['PRECIO_ALUM_IMP_t']
})

In [7]:
df_al.drop(df.index[0], inplace=True)

In [8]:
n_obs=6
X_train, X_test = df_al[0:-n_obs], df_al[-n_obs:]
print(X_train.shape, X_test.shape)
res = X_train.describe()
print(res)

(256, 2) (6, 2)
       PRECIO_ALUM_IMP  PRECIO_ALUM_IMP_t
count       256.000000         256.000000
mean       3602.542010           0.576734
std         551.658320           9.482653
min        1900.332225         -36.667765
25%        3208.033400          -5.247933
50%        3542.143114           0.562302
75%        3969.869367           6.001311
max        5352.655500          51.732716


In [9]:
from statsmodels.tsa.arima.model import ARIMA
model = ARIMA(X_train['PRECIO_ALUM_IMP'], order=(1,0,0)) # p, d, q deben ser números enteros
results = model.fit()

C:\Users\nef_m\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\nef_m\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\nef_m\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


In [10]:
print(X_test['PRECIO_ALUM_IMP'].iloc[:1])
print(X_test['PRECIO_ALUM_IMP'].iloc[:3])

FECHA_IN
2023-06-01    3958.406777
Name: PRECIO_ALUM_IMP, dtype: float64
FECHA_IN
2023-06-01    3958.406777
2023-07-01    4107.780336
2023-08-01    3930.282100
Name: PRECIO_ALUM_IMP, dtype: float64


In [11]:
forecast = results.forecast(steps=6)
print(forecast)
print(X_test['PRECIO_ALUM_IMP'])


2023-06-01    4022.606099
2023-07-01    3955.239742
2023-08-01    3898.677040
2023-09-01    3851.185394
2023-10-01    3811.310063
2023-11-01    3777.829605
Freq: MS, Name: predicted_mean, dtype: float64
FECHA_IN
2023-06-01    3958.406777
2023-07-01    4107.780336
2023-08-01    3930.282100
2023-09-01    4112.954174
2023-10-01    3830.759747
2023-11-01    4666.659224
Name: PRECIO_ALUM_IMP, dtype: float64


In [17]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def calculate_metrics(y_true, y_pred):
    # Calcular MSE
    mse = mean_squared_error(y_true, y_pred)
    
    # Calcular RMSE
    rmse = np.sqrt(mse)
    
    # Calcular MAE
    mae = mean_absolute_error(y_true, y_pred)
    
    # Calcular MAPE
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    # Calcular R^2
    r2 = r2_score(y_true, y_pred)
    
    return mse, rmse, mae, mape, r2

In [18]:
data = {
    'SERIE':[],
    'MSE':[],
    'RMSE':[],
    'MAE':[],
    'MAPE':[],
    'R2':[]
}
df_medidas_1 = pd.DataFrame(data)
df_medidas_3 = pd.DataFrame(data)
df_medidas_6 = pd.DataFrame(data)

    #medidas a 6 meses        
mse, rmse, mae, mape, r2 = calculate_metrics(X_test['PRECIO_ALUM_IMP'], forecast)
datos_6 = pd.Series({'SERIE': 'AR1', 'MSE': mse, 'RMSE': rmse, 'MAE': mae, 'MAPE': mape,'R2':r2})
    #medidas a 3 meses
mse, rmse, mae, mape, r2 = calculate_metrics(X_test['PRECIO_ALUM_IMP'][:3], forecast[:3])
datos_3 = pd.Series({'SERIE': 'AR1', 'MSE': mse, 'RMSE': rmse, 'MAE': mae, 'MAPE': mape,'R2':r2})
    #medidas a 1 mes
mse, rmse, mae, mape, r2 = calculate_metrics(X_test['PRECIO_ALUM_IMP'][:1], forecast[:1])
datos_1 = pd.Series({'SERIE': 'AR1', 'MSE': mse, 'RMSE': rmse, 'MAE': mae, 'MAPE': mape,'R2':r2})
#
df_medidas_1.loc[len(df_medidas_1)] = datos_1
df_medidas_3.loc[len(df_medidas_3)] = datos_3
df_medidas_6.loc[len(df_medidas_6)] = datos_6

df_medidas_1.to_excel('C:/Users/nef_m/OneDrive/Desktop/DATOS TESIS MGO/medidas AR_1.xlsx')
df_medidas_3.to_excel('C:/Users/nef_m/OneDrive/Desktop/DATOS TESIS MGO/medidas AR_3.xlsx')
df_medidas_6.to_excel('C:/Users/nef_m/OneDrive/Desktop/DATOS TESIS MGO/medidas AR_6.xlsx')

C:\Users\nef_m\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:1187: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [19]:
data = {
    'SERIE':[],
    'MSE':[],
    'RMSE':[],
    'MAE':[],
    'MAPE':[],
    'R2':[]
}
df_medidas_1 = pd.DataFrame(data)
df_medidas_3 = pd.DataFrame(data)
df_medidas_6 = pd.DataFrame(data)

    #medidas a 6 meses        
mse, rmse, mae, mape, r2 = calculate_metrics(X_test['PRECIO_ALUM_IMP'], forecast_)
datos_6 = pd.Series({'SERIE': 'MA1', 'MSE': mse, 'RMSE': rmse, 'MAE': mae, 'MAPE': mape,'R2':r2})
    #medidas a 3 meses
mse, rmse, mae, mape, r2 = calculate_metrics(X_test['PRECIO_ALUM_IMP'][:3], forecast_[:3])
datos_3 = pd.Series({'SERIE': 'MA1', 'MSE': mse, 'RMSE': rmse, 'MAE': mae, 'MAPE': mape,'R2':r2})
    #medidas a 1 mes
mse, rmse, mae, mape, r2 = calculate_metrics(X_test['PRECIO_ALUM_IMP'][:1], forecast_[:1])
datos_1 = pd.Series({'SERIE': 'MA1', 'MSE': mse, 'RMSE': rmse, 'MAE': mae, 'MAPE': mape,'R2':r2})
#
df_medidas_1.loc[len(df_medidas_1)] = datos_1
df_medidas_3.loc[len(df_medidas_3)] = datos_3
df_medidas_6.loc[len(df_medidas_6)] = datos_6

df_medidas_1.to_excel('C:/Users/nef_m/OneDrive/Desktop/DATOS TESIS MGO/medidas MA_1.xlsx')
df_medidas_3.to_excel('C:/Users/nef_m/OneDrive/Desktop/DATOS TESIS MGO/medidas MA_3.xlsx')
df_medidas_6.to_excel('C:/Users/nef_m/OneDrive/Desktop/DATOS TESIS MGO/medidas MA_6.xlsx')

C:\Users\nef_m\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:1187: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [13]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score

# Calcular el Error Cuadrático Medio (MSE)
mse = mean_squared_error(X_test['PRECIO_ALUM_IMP'], forecast)
# Calcular la Raíz del Error Cuadrático Medio (RMSE)
rmse = np.sqrt(mse)
# Calcular el Error Absoluto Medio (MAE)
mae = mean_absolute_error(X_test['PRECIO_ALUM_IMP'], forecast)
# Calcular el Error Porcentual Absoluto Medio (MAPE)
mape = mean_absolute_percentage_error(X_test['PRECIO_ALUM_IMP'], forecast)
# Calcular el Coeficiente de Determinación (R2)
r2 = r2_score(X_test['PRECIO_ALUM_IMP'], forecast)
medidas = {
    'SERIE': 'AR',
    'MSE': mse, 
    'RMSE': rmse, 
    'MAE': mae, 
    'MAPE': mape, 
    'R2': r2,
}
print(medidas)
# Convertir el diccionario en un DataFrame de pandas
df_medidas = pd.DataFrame(medidas, index=[0])
df_medidas_3 = pd.DataFrame(medidas, index=[0])
df_medidas_1 = pd.DataFrame(medidas, index=[0])

df_medidas.to_excel('C:/Users/nef_m/OneDrive/Desktop/DATOS TESIS MGO/medidas AR - 6.xlsx')
mse_3 = mean_squared_error(X_test['PRECIO_ALUM_IMP'][:3], forecast[:3])
rmse_3 = np.sqrt(mse_3)
mae_3 = mean_absolute_error(X_test['PRECIO_ALUM_IMP'][:3], forecast[:3])
mape_3 = mean_absolute_percentage_error(X_test['PRECIO_ALUM_IMP'][:3], forecast[:3])
r2_3 = r2_score(X_test['PRECIO_ALUM_IMP'][:3], forecast[:3])
# Convertir el diccionario en un DataFrame de pandas
df_medidas = pd.DataFrame(medidas, index=[0])
medidas_3 = {
    'SERIE': 'AR',
    'MSE': mse_3, 
    'RMSE': rmse_3, 
    'MAE': mae_3, 
    'MAPE': mape_3, 
    'R2': r2_3,
}
print(medidas_3)
df_medidas_3.to_excel('C:/Users/nef_m/OneDrive/Desktop/DATOS TESIS MGO/medidas AR - 3.xlsx')

mse_1 = mean_squared_error(X_test['PRECIO_ALUM_IMP'][:1], forecast[:1])
rmse_1 = np.sqrt(mse_1)
mae_1 = mean_absolute_error(X_test['PRECIO_ALUM_IMP'][:1], forecast[:1])
mape_1 = mean_absolute_percentage_error(X_test['PRECIO_ALUM_IMP'][:1], forecast[:1])
r2_1 = r2_score(X_test['PRECIO_ALUM_IMP'][:1], forecast[:1])

medidas_1 = {
    'SERIE': 'AR',
    'MSE': mse_1, 
    'RMSE': rmse_1, 
    'MAE': mae_1, 
    'MAPE': mape_1, 
    'R2': r2_1,
}
print(medidas_1)
df_medidas_1.to_excel('C:/Users/nef_m/OneDrive/Desktop/DATOS TESIS MGO/medidas AR - 1.xlsx')

{'SERIE': 'AR', 'MSE': 147884.7236548404, 'RMSE': 384.5578287525043, 'MAE': 236.39884313385974, 'MAPE': 0.053430073288283274, 'R2': -1.0044631040238223}
{'SERIE': 'AR', 'MSE': 9463.021824219904, 'RMSE': 97.27806445555906, 'MAE': 82.7816584927911, 'MAPE': 0.02046481753855847, 'R2': -0.5595793754671865}
{'SERIE': 'AR', 'MSE': 4121.552930185636, 'RMSE': 64.19932188259963, 'MAE': 64.19932188259963, 'MAPE': 0.01621847513369986, 'R2': nan}


C:\Users\nef_m\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:1187: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [ ]:
print(forecast)


In [16]:
model_ = ARIMA(X_train['PRECIO_ALUM_IMP'], order=(0,0,1)) # p, d, q deben ser números enteros
results_ = model_.fit()
forecast_ = results_.forecast(steps=6)

data = {
    'SERIE':[],
    'MSE':[],
    'RMSE':[],
    'MAE':[],
    'MAPE':[],
    'R2':[],
}

df_medidas_1 = pd.DataFrame(data)
df_medidas_3 = pd.DataFrame(data)
df_medidas_6 = pd.DataFrame(data)

print(X_test['PRECIO_ALUM_IMP'])
# Calcular el Error Cuadrático Medio (MSE)
mse = mean_squared_error(X_test['PRECIO_ALUM_IMP'][:6], forecast_[:6])
# Calcular la Raíz del Error Cuadrático Medio (RMSE)
rmse = np.sqrt(mse)
# Calcular el Error Absoluto Medio (MAE)
mae = mean_absolute_error(X_test['PRECIO_ALUM_IMP'][:6], forecast_[:6])
# Calcular el Error Porcentual Absoluto Medio (MAPE)
mape = mean_absolute_percentage_error(X_test['PRECIO_ALUM_IMP'][:6], forecast_[:6])
# Calcular el Coeficiente de Determinación (R2)
r2 = r2_score(X_test['PRECIO_ALUM_IMP'][:6], forecast_[:6])


df_medidas_6.to_excel('C:/Users/nef_m/OneDrive/Desktop/DATOS TESIS MGO/medidas MA - 6.xlsx')

mse_3 = mean_squared_error(X_test['PRECIO_ALUM_IMP'][:3], forecast_[:3])
rmse_3 = np.sqrt(mse)
mae_3 = mean_absolute_error(X_test['PRECIO_ALUM_IMP'][:3], forecast_[:3])
mape_3 = mean_absolute_percentage_error(X_test['PRECIO_ALUM_IMP'][:3], forecast_[:3])
r2_3 = r2_score(X_test['PRECIO_ALUM_IMP'][:3], forecast_[:3])
# Convertir el diccionario en un DataFrame de pandas
df_medidas = pd.DataFrame(medidas, index=[0])
medidas_3 = {
    'SERIE': 'MA',
    'MSE': mse_3, 
    'RMSE': rmse_3, 
    'MAE': mae_3, 
    'MAPE': mape_3, 
    'R2': r2_3,
}

df_medidas_3.to_excel('C:/Users/nef_m/OneDrive/Desktop/DATOS TESIS MGO/medidas MA - 3.xlsx')

mse_1 = mean_squared_error(X_test['PRECIO_ALUM_IMP'][:1], forecast_[:1])
rmse_1 = np.sqrt(mse)
mae_1 = mean_absolute_error(X_test['PRECIO_ALUM_IMP'][:1], forecast_[:1])
mape_1 = mean_absolute_percentage_error(X_test['PRECIO_ALUM_IMP'][:1], forecast_[:1])
r2_1 = r2_score(X_test['PRECIO_ALUM_IMP'][:1], forecast_[:1])

medidas_1 = {
    'SERIE': 'MA',
    'MSE': mse_1, 
    'RMSE': rmse_1, 
    'MAE': mae_1, 
    'MAPE': mape_1, 
    'R2': r2_1,
}
df_medidas_1.to_excel('C:/Users/nef_m/OneDrive/Desktop/DATOS TESIS MGO/medidas MA - 1.xlsx')

C:\Users\nef_m\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\nef_m\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\nef_m\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


FECHA_IN
2023-06-01    3958.406777
2023-07-01    4107.780336
2023-08-01    3930.282100
2023-09-01    4112.954174
2023-10-01    3830.759747
2023-11-01    4666.659224
Name: PRECIO_ALUM_IMP, dtype: float64


C:\Users\nef_m\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:1187: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


In [ ]:
print(forecast_)
